# Testing wrap_openai + openai responses api 

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

In [2]:
import openai
from langsmith import traceable
from langsmith.wrappers import wrap_openai

client = wrap_openai(openai.Client())

from openai import OpenAI

response = client.responses.create(
  model="gpt-4.1",
  input="Tell me a three sentence bedtime story about a unicorn."
)

print(response)


Response(id='resp_04b2c2ac9c9601a90068e451eaa79c81a28fc39ef257703841', created_at=1759793642.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4.1-2025-04-14', object='response', output=[ResponseOutputMessage(id='msg_04b2c2ac9c9601a90068e451eb60c081a2a53cabb14d31bebd', content=[ResponseOutputText(annotations=[], text='Once upon a time, a gentle unicorn named Luna danced under a starlit sky, her silver horn shimmering with magic. She found a lonely puppy and, with a touch of her horn, filled his dreams with joy and adventure. Every night after, the two friends explored sparkling meadows together, safe and happy beneath the moon’s soft glow.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[], top_p=1.0, background=False, conversation=None, max_output_tokens=None, max_tool_calls=None, previous_response_id=None, prompt=None, prompt_cache_key=Non

# P0

Input rendering does not look right: https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&tab=0&peek=6f9488d7-d727-4571-86b0-7472602c9243&peeked_trace=6f9488d7-d727-4571-86b0-7472602c9243

Input contains this openai.Omit object. Should we be dropping these on ingest?

Padding seems off with Additional fields

In [27]:
# web search tool

response = client.responses.create(
    model="gpt-4o",  # or another supported model
    input="What's the latest news about AI?",
    tools=[
        {
            "type": "web_search"
        }
    ]
)

In [3]:
import base64

from IPython.display import Image, display

# Display the image from the provided URL
url = "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"
display(Image(url=url, width=400))

response_multimodal = client.responses.create(
    model="gpt-4o",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": 
                 "Come up with keywords related to the image, and search on the web using the search tool for any news related to the keywords"
                 ", summarize the findings and cite the sources."},
                {"type": "input_image", "image_url": "https://upload.wikimedia.org/wikipedia/commons/thumb/1/15/Cat_August_2010-4.jpg/2880px-Cat_August_2010-4.jpg"}
            ]
        }
    ],
    tools=[
        {"type": "web_search"}
    ]
)


In [ ]:
import json

# 1. Define a list of callable tools for the model
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get today's horoscope for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "An astrological sign like Taurus or Aquarius",
                },
            },
            "required": ["sign"],
        },
    },
        {
        "type": "function",
        "name": "get_retrograde_planets",
        "description": "Get today's retrograde planets for an astrological sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "sign": {
                    "type": "string",
                    "description": "Curernt date",
                },
            },
            "required": ["date"],
        },
    },
]

def get_retrograde_planets(date):
    return f"Retrograde planets for {date}: Mercury, Venus, and Mars."

def get_horoscope(sign):
    return f"{sign}: Next Tuesday you will befriend a baby otter."

# Create a running input list we will add to over time
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

# 2. Prompt the model with tools defined
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Save function call outputs for subsequent requests
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # 3. Execute the function logic for get_horoscope
            horoscope = get_horoscope(json.loads(item.arguments))
            
            # 4. Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

print("Final input:")
print(input_list)

response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)

# 5. The model should be able to give a response!
print("Final output:")
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

# P0

UI doesnt show the tool that was called in the "Tools" section. IIRC we show a "Called" (or something similar to this) tag normally 

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&tab=0&peek=ca327adc-95e6-4ef7-bc0a-03d817ba9deb&peeked_trace=ca327adc-95e6-4ef7-bc0a-03d817ba9deb

In [ ]:

resp = client.responses.create(
    model="gpt-5",
    tools=[
        {
            "type": "mcp",
            "server_label": "dmcp",
            "server_description": "A Dungeons and Dragons MCP server to assist with dice rolling.",
            "server_url": "https://dmcp-server.deno.dev/sse",
            "require_approval": "never",
        },
    ],
    input="Roll 2d4+1",
)

print(resp.output_text)

In [ ]:

instructions = """
You are a personal math tutor. When asked a math question, 
write and run code using the python tool to answer the question.
"""

resp = client.responses.create(
    model="gpt-4.1",
    tools=[
        {
            "type": "code_interpreter",
            "container": {"type": "auto"}
        }
    ],
    instructions=instructions,
    input="I need to solve the equation 3x + 11 = 14. Can you help me?",
)

print(resp.output)

# P1

code interpreter tool doesnt render under "Tools" section 
https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&tab=0&peek=83371525-e6fa-44f5-98fe-beccacfc4813&peeked_trace=83371525-e6fa-44f5-98fe-beccacfc4813

In [ ]:
## image generation tool 

response = client.responses.create(
    model="gpt-5",
    input="Generate an image of gray tabby cat hugging an otter with an orange scarf",
    tools=[{"type": "image_generation"}],
)

# P2

Mostly renders well. There's an empty AI messages that shows up though

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&tab=0&peek=210da83b-c7b6-4a1e-9816-a0772d454fd9&peeked_trace=210da83b-c7b6-4a1e-9816-a0772d454fd9

In [37]:
from pydantic import BaseModel

client = wrap_openai(openai.Client())

class CalendarEvent(BaseModel):
    name: str
    date: str
    participants: list[str]

response = client.responses.parse(
    model="gpt-4o-2024-08-06",
    input=[
        {"role": "system", "content": "Extract the event information."},
        {
            "role": "user",
            "content": "Alice and Bob are going to a science fair on Friday.",
        },
    ],
    text_format=CalendarEvent,
)

event = response.output_parsed

# P1 

We dont show text_format as the output schema for the llm call 

https://smith.langchain.com/o/ebbaf2eb-769b-4505-aca2-d11de10372a4/projects/p/5b35f59d-3a34-49d1-b6bb-b31d3cf45367?columnVisibilityModel=%7B%22feedback_stats%22%3Afalse%2C%22reference_example%22%3Afalse%2C%22start_time%22%3Atrue%2C%22inputs%22%3Atrue%2C%22outputs%22%3Atrue%2C%22error%22%3Atrue%2C%22latency%22%3Atrue%2C%22in_dataset%22%3Afalse%2C%22last_queued_at%22%3Afalse%2C%22total_tokens%22%3Atrue%2C%22total_cost%22%3Atrue%2C%22first_token_time%22%3Afalse%2C%22tags%22%3Atrue%2C%22metadata%22%3Atrue%2C%22reference_example_id%22%3Atrue%7D&timeModel=%7B%22duration%22%3A%227d%22%7D&runtab=0&tab=0&peek=695a031e-d7dc-45aa-a756-553384fea5ac&peeked_trace=695a031e-d7dc-45aa-a756-553384fea5ac
